# Top shooters by FG% in the 2019-20 regular season
---

This notebook will run an analysis on the FG% per player to determine and display the most efficient scorers by zone.

+ To remove players with low attempts, we will consider only players in the top x quantile of attempts in the zone. Default is .5 but can be changed below.
+ Plotting players only works when there's a valid secrets file containing a Google access key
+ Scroll to the bottom for the final result!

---
## User input
`min_quant`: minimum quantile of shots-attempted (for the zone) that a player must be in to be considered in the analysis

`season` = single season in `xxxx-xx` format, eg. `2019-20`

In [41]:
min_quant = .5
season = "2019-20"

No user input needed below this line

---

In [64]:
import matplotlib.pyplot as plt
import pandas as pd
import json
from Naked.toolshed.shell import muterun_js
from PIL import ( Image, ImageDraw, ImageFont )
from google_images_search import GoogleImagesSearch
import datetime

In [31]:
response = muterun_js('importShotsData_python.js', season)

if response.exitcode == 0:
    file_name = response.stdout.decode('utf-8').replace('\n','')
else:
    print("call failed with err: " + response.stderr)

In [46]:
with open(file_name) as f:
    df = json.load(f)
    
shots = df['shot_Chart_Detail']
pd_shots = pd.DataFrame(shots)[['playerName','shotZoneArea','shotZoneBasic','shotAttemptedFlag','shotMadeFlag']]

# Aggregate data
player_agg = pd_shots.groupby(['shotZoneArea','shotZoneBasic','playerName'])[['shotAttemptedFlag','shotMadeFlag']].agg('sum')
eligibility = pd.DataFrame(player_agg.reset_index(level='playerName').groupby(['shotZoneArea','shotZoneBasic'])['shotAttemptedFlag'].quantile(q=min_quant))

eligibility = eligibility.rename(columns={'shotAttemptedFlag': 'shotCriteria'})
player_agg = player_agg.reset_index(level='playerName')

eligibility = eligibility.join(player_agg, on=['shotZoneArea','shotZoneBasic'])
candidates = eligibility.query('shotAttemptedFlag > shotCriteria & shotMadeFlag > 0')
candidates['fgPct'] = candidates['shotMadeFlag'] / candidates['shotAttemptedFlag']

candidates['rank'] = candidates.groupby(['shotZoneArea','shotZoneBasic'])['fgPct'].rank(method='dense', ascending=False)
candidates = candidates.reset_index(level=['shotZoneArea','shotZoneBasic'])

<ipython-input-46-c499f6cf4745>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidates['fgPct'] = candidates['shotMadeFlag'] / candidates['shotAttemptedFlag']
<ipython-input-46-c499f6cf4745>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidates['rank'] = candidates.groupby(['shotZoneArea','shotZoneBasic'])['fgPct'].rank(method='dense', ascending=False)


In [47]:
pos_lookup = {"Back Court(BC)": {"Above the Break 3": (0.5, 0.15),
                                 "Backcourt": (0.5, 0),
                                },
              "Center(C)": {"Above the Break 3": (0.5, 0.45),
                            "In The Paint (Non-RA)": (0.5, 0.7),
                            "Mid-Range": (0.5, 0.55),
                            "Restricted Area": (0.5, 0.9),
                           },
              "Left Side Center(LC)": {"Above the Break 3": (0.2, 0.55),
                                       "Mid-Range": (0.25, 0.65),
                                      },
              "Left Side(L)": {"In The Paint (Non-RA)":(0.4, 0.85),
                               "Mid-Range": (0.25, 0.78),
                               "Left Corner 3": (0.1, 0.9),
                              },
              "Right Side Center(RC)": {"Above the Break 3": (0.8, 0.55),
                                        "Mid-Range": (0.75, 0.65),
                                       },
              "Right Side(R)": {"In The Paint (Non-RA)":(0.6, 0.85),
                               "Mid-Range": (0.75, 0.78),
                               "Right Corner 3": (0.9, 0.9),
                              },
                  
              }

In [92]:
def get_headshot(playerName):
    try:
        image = Image.open('./img/{}.png'.format(playerName)).convert("RGBA")
        return image
    except:
        _search_params = {
        'q': '{} Headshot'.format(playerName),
        'num': 1,
        'fileType': 'png',
        }

        gis.search(search_params=_search_params, path_to_dir='./img/',\
                   custom_image_name=playerName, width=int(WIDTH/10), height=int(WIDTH/10))
    
    return Image.open('./img/{}.png'.format(playerName)).convert("RGBA")

In [93]:
with open('./secrets.json') as s:
    secrets = json.load(s)
key = secrets['googleKey']
cx = secrets['googleSearchCX']

gis = GoogleImagesSearch(key, cx)

In [94]:
img = Image.open('./img/darkCourt.png')

WIDTH, HEIGHT = img.getdata().size
img.resize((int(WIDTH*1.5), int(HEIGHT*1.5)))

draw = ImageDraw.Draw(img)
font = ImageFont.load_default()

for index, row in candidates.query('rank == 1').iterrows():
    msg = "{}\n{}% from {}".format(row['playerName'], round(row['fgPct']*100,2), row['shotAttemptedFlag'])
    
    playerImg = get_headshot(row['playerName'])
    
    w, h = draw.textsize(msg)
    w, h = w/2, h/2
    
    x, y = pos_lookup[row['shotZoneArea']][row['shotZoneBasic']]
    x = max((x*WIDTH) - w, 0)
    y = max((y*HEIGHT) - h, 0)
    
    draw.text(( x , int(y + WIDTH/10) ), msg, (255,255,255),font=font)
    img.paste(playerImg, (int(x),int(y)), playerImg)

img_name = './img/topFGpct_{}Percentile_{}.png'.format(min_quant, str(datetime.datetime.now())[0:10])
img.save(img_name, "PNG")

---
# The result
See the players with the top FG% per zone:

![img](img/topFGpct_0.5Percentile_2020-08-21.png)

In [95]:
candidates.query('rank==1')

,shotZoneArea,shotZoneBasic,shotCriteria,playerName,shotAttemptedFlag,shotMadeFlag,fgPct,rank
0,Back Court(BC),Above the Break 3,1.0,Chris Paul,3,1,0.333333,1.0
3,Back Court(BC),Backcourt,1.0,Svi Mykhailiuk,3,1,0.333333,1.0
164,Center(C),Above the Break 3,20.0,Meyers Leonard,42,23,0.547619,1.0
426,Center(C),In The Paint (Non-RA),32.5,Nikola Jokic,251,154,0.613546,1.0
628,Center(C),Mid-Range,7.0,Matt Thomas,8,6,0.750000,1.0
883,Center(C),Restricted Area,82.0,Nerlens Noel,174,147,0.844828,1.0
1059,Left Side Center(LC),Above the Break 3,30.5,Justin Holiday,86,43,0.500000,1.0
1262,Left Side Center(LC),Mid-Range,5.0,Jonas Valanciunas,7,6,0.857143,1.0
1399,Left Side(L),In The Paint (Non-RA),3.0,Harrison Barnes,8,7,0.875000,1.0
1587,Left Side(L),Left Corner 3,14.0,Joe Harris,37,25,0.675676,1.0
